In [1]:
# import os
# import json
# import pickle
import random
import warnings
warnings.filterwarnings('ignore')
import spacy
# from spacy.lang.en.stop_words import STOP_WORDS
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from utils import Data
data_processor = Data(root_path="../")

from collections import namedtuple
from tqdm import tqdm

print("Data files: ")
for file in data_processor.data_dict.keys():
    print(file)

Data files: 
collection.sampled.tsv
train_sample_queries.tsv
train_sample_passv2_qrels.tsv
val_2021_53_queries.tsv
val_2021_passage_top100.txt
val_2021.qrels.pass.final.txt
test_2022_76_queries.tsv
test_2022_passage_top100.txt
test_2022.qrels.pass.withDupes.txt


In [2]:
data_processor.read_in_memory()

正在处理文件collection.sampled.tsv 读取文件的格式为('pid', 'passage')
正在处理文件train_sample_queries.tsv 读取文件的格式为('qid', 'query')
正在处理文件train_sample_passv2_qrels.tsv 读取文件的格式为('qid', 'mark', 'pid', 'rating')
正在处理文件val_2021_53_queries.tsv 读取文件的格式为('qid', 'query')
正在处理文件val_2021_passage_top100.txt 读取文件的格式为('qid', 'mark', 'pid', 'rank', 'score', 'sys_id')
正在处理文件val_2021.qrels.pass.final.txt 读取文件的格式为('qid', 'mark', 'pid', 'rating')
正在处理文件test_2022_76_queries.tsv 读取文件的格式为('qid', 'query')
正在处理文件test_2022_passage_top100.txt 读取文件的格式为('qid', 'mark', 'pid', 'rank', 'score', 'sys_id')
正在处理文件test_2022.qrels.pass.withDupes.txt 读取文件的格式为('qid', 'mark', 'pid', 'rating')


In [3]:
# read data from memory by calling data_processor.dataset with following keys
for f in data_processor.dataset.keys():
    print(f)

collection.sampled
train_sample_queries
train_sample_passv2_qrels
val_2021_53_queries
val_2021_passage_top100
val_2021.qrels.pass.final
test_2022_76_queries
test_2022_passage_top100
test_2022.qrels.pass.withDupes


In [4]:
set_passage_id = set(data_processor.dataset['collection.sampled'].keys())
print("Total number of passages: ", len(set_passage_id))

Total number of passages:  126799


In [5]:
example_passage_id = random.choice(list(set_passage_id))
example_passage = data_processor.dataset['collection.sampled'][example_passage_id]['passage']
doc = nlp(example_passage)
sentences = [i.text for i in doc.sents]
sentences

["'When an action potential causes cells to contract, calcium is released from the sarcoplasmic reticulum of the cells as well as the T tubules.",
 'The calcium release triggers sliding of the actin and myosin fibrils leading to contraction.',
 "A plentiful supply of mitochondria provide the energy for the contractions.'"]

In [6]:
doc = nlp(sentences[0])
displacy.render(doc, style='dep', jupyter=True)

In [7]:
for token in doc:
    if token.is_stop or token.is_punct:
        continue
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.head.text)

action action NOUN NN compound potential
potential potential NOUN NN nsubj causes
causes cause VERB VBZ advcl released
cells cell NOUN NNS nsubj contract
contract contract NOUN NN ccomp causes
calcium calcium NOUN NN nsubjpass released
released release VERB VBN ROOT released
sarcoplasmic sarcoplasmic ADJ JJ amod reticulum
reticulum reticulum NOUN NN pobj from
cells cell NOUN NNS pobj of
T t NOUN NN compound tubules
tubules tubule NOUN NNS appos reticulum


In [8]:
# with doc.retokenize() as retokenizer:
#     retokenizer.merge(doc[1:3], attrs={"LEMMA": "quick_brown_fox"})
# displacy.render(doc, style='dep', jupyter=True)

In [9]:
# 生成依存树

class Node:
    def __init__(self, text, parent, children, dep):
        self.text = text
        self.parent = parent
        self.children = children
        self.dep = dep

nodes = {}
for token in doc:
    node = Node(
        text=token.text,
        parent=None,
        children=set(),
        dep=token.dep_
    )
    nodes[token.text] = node

for token in doc:
    text, head, dep, head_dep = token.text, token.head.text, token.dep_, token.head.dep_
    nodes[text].parent = nodes[head]
    nodes[head].children.add(nodes[text])

for text, node in nodes.items():
    if node.text == node.parent.text:
        root = node
        break

root.text

'released'